In [ ]:
import sys, os
import time
from pathlib import Path
import motornet as mn
import torch as th
import numpy as np
import matplotlib.pyplot as plt
import json
from task import CentreOutFF
from policy import Policy
from plotting import plot_simulations, plot_training_log, plot_activation
from utils import calculate_angles_between_vectors, create_directory
device = th.device("cpu")

In [ ]:
# Function to load (or make new) Environment

def load_env(task, cfg=None):

    if cfg is None:

        name = 'env'

        action_noise         = 1e-4
        proprioception_noise = 1e-3
        vision_noise         = 1e-4
        vision_delay         = 0.05
        proprioception_delay = 0.02

        # Define task and the effector
        effector = mn.effector.RigidTendonArm26(muscle=mn.muscle.RigidTendonHillMuscle())

        max_ep_duration = 1.0
    else:
        name = cfg['name']
        # effector
        muscle_name = cfg['effector']['muscle']['name']
        timestep = cfg['effector']['dt']
        muscle = getattr(mn.muscle,muscle_name)()
        effector = mn.effector.RigidTendonArm26(muscle=muscle,timestep=timestep) 

        # delay
        proprioception_delay = cfg['proprioception_delay']*cfg['dt']
        vision_delay = cfg['vision_delay']*cfg['dt']

        # noise
        action_noise = cfg['action_noise'][0]
        proprioception_noise = cfg['proprioception_noise'][0]
        vision_noise = cfg['vision_noise'][0]

        # initialize environment
        max_ep_duration = cfg['max_ep_duration']


    env = task(effector=effector,max_ep_duration=max_ep_duration,name=name,
               action_noise=action_noise,proprioception_noise=proprioception_noise,
               vision_noise=vision_noise,proprioception_delay=proprioception_delay,
               vision_delay=vision_delay)

    return env

In [ ]:
# Function to Train a Network

def train(model_num, ff_coefficient, phase, condition='train', directory_name=None, n_batch=None, batch_size=None, interval=100):

  output_folder = create_directory(directory_name=directory_name)
  model_name = "model{:02d}".format(model_num)
  device = th.device("cpu")

  # Set configuaration and network
  if phase>=1:
    print("Training phase {}...".format(phase))
    # load config and weights from the previous phase
    weight_file = list(Path(output_folder).glob(f'{model_name}_phase={phase-1}_*_weights'))[0]
    cfg_file = list(Path(output_folder).glob(f'{model_name}_phase={phase-1}_*_cfg.json'))[0]

    # load configuration
    with open(cfg_file,'r') as file:
      cfg = json.load(file)

    # environment and network
    env = load_env(CentreOutFF,cfg)
    policy = Policy(env.observation_space.shape[0], 128, env.n_muscles, device=device, freeze_output_layer=True)
    policy.load_state_dict(th.load(weight_file))

  else:
    # environment and network
    env = load_env(CentreOutFF)
    policy = Policy(env.observation_space.shape[0], 128, env.n_muscles, device=device)
  
  if condition=='growing_up': 
    optimizer = th.optim.Adam(policy.parameters(), lr=0.001,eps=1e-7)
    catch_trial_perc = 50

  else: # for training use biologily plausible optimizer
    optimizer = th.optim.SGD(policy.parameters(), lr=0.001)
    catch_trial_perc = 50

  # define an l1 loss function
  def l1(x, y):
    """L1 loss"""
    return th.mean(th.sum(th.abs(x - y), dim=-1))

  # Train network
  overall_losses = []
  input_losses = []
  position_losses = []
  angle_losses = []
  muscle_losses = []
  hidden_losses = []

  batch_time = 0.0
  forward_time = 0.0
  loss_time = 0.0
  backward_time = 0.0
  optimstep_time = 0.0

  for batch in range(n_batch):

    batch_start_time = time.time()

    # Run episode
    h = policy.init_hidden(batch_size = batch_size)

    obs, info = env.reset(condition        = 'train', 
                          catch_trial_perc = catch_trial_perc, 
                          ff_coefficient   = ff_coefficient, 
                          options          = {'batch_size':batch_size})
    terminated = False

    # initial positions and targets
    xy = []
    tg = []
    all_actions = []
    all_hidden = []
    all_muscle = []
    all_force = []

    # simulate whole episode
    while not terminated:  # will run until `max_ep_duration` is reached
      action, h = policy(obs,h)
      all_hidden.append(h[0,:,None,:])
      obs, _, terminated, _, info = env.step(action=action)
      xy.append(info['states']['cartesian'][:, None, :])  # trajectories
      tg.append(info["goal"][:, None, :])  # targets
      all_actions.append(action[:, None, :])
      all_muscle.append(info['states']['muscle'][:,0,None,:])
      all_force.append(info['states']['muscle'][:,6,None,:])

    # concatenate into a (batch_size, n_timesteps, xy) tensor
    xy = th.cat(xy, axis=1)
    tg = th.cat(tg, axis=1)
    
    all_actions = th.cat(all_actions, axis=1)
    all_muscle = th.cat(all_muscle, axis=1)
    all_hidden = th.cat(all_hidden, axis=1)
    all_force = th.cat(all_force, axis=1)

    forward_time += (time.time() - batch_start_time)
    loss_start_time = time.time()

    # CALCULATE LOSSES
    # input_loss
    input_loss = th.sqrt(th.sum(th.square(policy.gru.weight_ih_l0)))
    # muscle_loss
    max_iso_force_n = env.muscle.max_iso_force / th.mean(env.muscle.max_iso_force) 
    y = all_muscle * max_iso_force_n
    muscle_loss = th.mean(th.square(y))
    # hidden_loss
    y = all_hidden
    dy = th.diff(y,axis=1)/env.dt
    hidden_loss = th.mean(th.square(y))+0.05*th.mean(th.square(dy))
    # position_loss
    position_loss = l1(xy[:,:,0:2], tg)

    loss = 1e-6*input_loss + 20*muscle_loss + 0.1*hidden_loss + 2*position_loss
    
    loss_time += (time.time() - loss_start_time)
    backward_start_time = time.time()

    # backward pass & update weights
    optimizer.zero_grad() 
    loss.backward()
    th.nn.utils.clip_grad_norm_(policy.parameters(), max_norm=1.)  # important!

    backward_time += (time.time() - backward_start_time)
    optimstep_start_time = time.time()

    optimizer.step()
    overall_losses.append(loss.item())
    input_losses.append(input_loss.item())
    hidden_losses.append(hidden_loss.item())
    muscle_losses.append(muscle_loss.item())
    position_losses.append(position_loss.item())

    optimstep_time += (time.time() - optimstep_start_time)
    batch_time += (time.time() - batch_start_time)

    if (batch % interval == 0) and (batch != 0):
        print("--------------------------------------------------------------------------------")
        print("Batch {}/{} Done, mean position loss: {}".format(batch, n_batch, sum(position_losses[-interval:])/interval))
        print(f"{batch_time:.3f} s: [ {forward_time/batch_time*100:.3f} %/fwd, {loss_time/batch_time*100:.3f} %/loss, {backward_time/batch_time*100:.3f} %/backward, {optimstep_time/batch_time*100:.3f} %/optimstep ]")

        # For saving model
        weight_file = os.path.join(output_folder, f"{model_name}_phase={phase}_FFCoef={ff_coefficient}_weights")
        log_file = os.path.join(output_folder, f"{model_name}_phase={phase}_FFCoef={ff_coefficient}_log.json")
        cfg_file = os.path.join(output_folder, f"{model_name}_phase={phase}_FFCoef={ff_coefficient}_cfg.json")

        # save model weights
        th.save(policy.state_dict(), weight_file)

        # save training history (log)
        with open(log_file, 'w') as file:
            json.dump({'overall_loss':overall_losses,
                    'muscle_loss':muscle_losses,
                    'hidden_loss':hidden_losses,
                    'position_loss':position_losses,
                    'input_loss':input_losses}, file)

        # save environment configuration dictionary
        cfg = env.get_save_config()
        with open(cfg_file, 'w') as file:
            json.dump(cfg,file)

  print("done.")


In [ ]:
# Function for Testing a Network

def test(cfg_file, weight_file, ff_coefficient=None):
  
  device = th.device("cpu")

  # load configuration
  with open(cfg_file,'r') as file:
    cfg = json.load(file)

  if ff_coefficient is None:
    ff_coefficient=cfg['ff_coefficient']
    
  # environment and network
  env = load_env(CentreOutFF, cfg)
  policy = Policy(env.observation_space.shape[0], 128, env.n_muscles, device=device)
  policy.load_state_dict(th.load(weight_file))
  
  batch_size = 8
  # initialize batch
  obs, info = env.reset(condition ='test',catch_trial_perc=0,options={'batch_size':batch_size},ff_coefficient=ff_coefficient)

  h = policy.init_hidden(batch_size=batch_size)
  terminated = False

  # initial positions and targets
  xy = []
  tg = []
  vel = []
  all_actions = []
  all_muscles = []
  all_hidden = []

  # simulate whole episode
  while not terminated:  # will run until `max_ep_duration` is reached
    all_hidden.append(h[0,:,None,:])
    action, h = policy(obs, h)
    obs, reward, terminated, truncated, info = env.step(action=action)  
    xy.append(info["states"]["fingertip"][:,None,:])  # trajectories
    tg.append(info["goal"][:,None,:])  # targets
    vel.append(info["states"]["cartesian"][:,None,2:])
    all_actions.append(action[:, None, :])    
    all_muscles.append(info['states']['muscle'][:,0,None,:])

  # concatenate into a (batch_size, n_timesteps, xy) tensor
  xy = th.detach(th.cat(xy, axis=1))
  tg = th.detach(th.cat(tg, axis=1))
  vel = th.detach(th.cat(vel, axis=1))
  all_hidden = th.detach(th.cat(all_hidden, axis=1))
  all_actions = th.detach(th.cat(all_actions, axis=1))
  all_muscles = th.detach(th.cat(all_muscles, axis=1))

  return xy, tg, all_hidden, all_muscles, vel

In [ ]:
# Training Parameters

model_num      = 0
directory_name = 'demo'
phase          = 0
condition      = 'growing_up'
ff_coefficient = 0.0
n_batch        = 30000
batch_size     = 1024
interval       = 1000


In [ ]:
# Train Network on point to point random targets

train(model_num      = model_num,
      ff_coefficient = ff_coefficient,
      phase          = phase,
      condition      = condition,
      directory_name = directory_name,
      n_batch        = n_batch,
      batch_size     = batch_size,
      interval       = interval)


In [ ]:
# Test Network

model_name = "model{:02d}".format(model_num)
data_dir = create_directory(directory_name=directory_name)
log_file = list(Path(data_dir).glob(f'{model_name}_phase={phase}_*_log.json'))[0]
weight_file = list(Path(data_dir).glob(f'{model_name}_phase={phase}_*_weights'))[0]
cfg_file = list(Path(data_dir).glob(f'{model_name}_phase={phase}_*_cfg.json'))[0]

In [ ]:
# Plot training log (loss over time)

log = json.load(open(log_file,'r'))
fig,ax = plot_training_log(log=log,loss_type='position_loss')
fig.savefig(os.path.join(data_dir,'training_log.png'),dpi=300)

In [ ]:
# Plot hand trajectories

xy, tg, all_hidden, all_muscles, vel = test(cfg_file, weight_file, ff_coefficient=0.0)
fig, ax  = plot_simulations(xy=xy, target_xy=tg, vel=None, figsize=(6,8))
fig.savefig(os.path.join(data_dir,'hand_paths.png'),dpi=300)

In [ ]:
# Plot activations

fig, ax = plot_activation(all_hidden, all_muscles, xy, tg)
fig.savefig(os.path.join(data_dir,'activations.png'),dpi=300)

In [ ]:
print(np.shape(log["overall_loss"]))